In [1]:
# file_path = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overconsistency_gpt4o_eval.txt"

# with open(file_path, "r") as file:
#     lines = file.readlines()

# #format the output, remove the \n at the end of each line
# new_lines = [x for x in lines if x != "\n"]

# s = {}

# for index in range(0,180):
#     add = 0
#     output = ""
#     while(add<6):
#         output += new_lines[index*6+add]
#         add += 1
#     s[str(index)] = output

# import json

# file_path = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overall_consistency_gpt4eval_results.json"

# with open(file_path, "r") as file:
#     data = json.load(file)
# for key in data.keys():
#     data[key] = data[key].replace('\n\n','\n')
# ss = s|data
# print(len(s.keys()))
# # 使用 json 保存字典
# with open(r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overall_consistency_gpt4eval_results.json", "w") as f:
#     json.dump(ss, f,indent=4)

In [2]:
dimension = 'object_class'
models = ['cogvideox5b','kling','gen3','lavie','pika','show1','videocrafter2']


In [3]:
# # 把gpt4o_eval的结果加入到annotation中
# import json
# import os


# file_path = "../GPT4o_eval_results/{}/{}_llmeval_eco.json".format(dimension,dimension)
# with open(file_path, "r") as file:
#     data = json.load(file)

# video_path ="../Human_anno/{}.json".format(dimension)
# with open(video_path, "r") as file:
#     annotations = json.load(file)

# for index,key in enumerate(data.keys()):
#     for line in data[key].split('\n'):
#         if line.split(' ')[0] != '-':
#             continue 
#         else:
#             try:
#                 score = int(line.split(' ')[2][0])
#             except:
#                 print(line,key)
#             # score = int(line.split(' ')[2][0])
#             model =line.split(' ')[1][:-1].lower().replace('**','')
#             annotations[int(key)]['gpt4o_eval'][model] = score

# with open(video_path, "w") as f:
#     json.dump(annotations, f,indent=4)

In [4]:
# # 5->3
# import json
# path ="../Human_anno/humananno_res/object_class_3.json"
# with open(path,'r') as f:
#     data = json.load(f)
# for i in range(len(data[dimension].keys())):
#     for model in models:
#         if data[dimension][str(i+1)][model] == 2:
#             data[dimension][str(i+1)][model] = 1
#         elif data[dimension][str(i+1)][model] == 3:
#             data[dimension][str(i+1)][model] = 2
#         elif  data[dimension][str(i+1)][model] == 4 or data[dimension][str(i+1)][model] == 5:
#              data[dimension][str(i+1)][model] = 3
# with open(path,'w') as f:
#     json.dump(data,f,indent=4)

In [5]:
# 把人类标注数据写入json文件 来源anno_res_anlysis.ipynb
import json

import pandas as pd
anno_path = '../Human_anno/humananno_res/object_class_lsy.json'

with open(anno_path, 'r') as f:
    anno_1 = json.load(f)

jsonpath = '../Human_anno/{}.json'.format(dimension)


with open(jsonpath, 'r') as f:
    oc = json.load(f)

for i in range(len(anno_1[dimension].keys())):
    for model in models:
        # oc[i]['human_anno'][model].insert(0, anno_1[dimension][str(i+1)][model])  # 在开头插入数据
        # oc[i]['human_anno'][model].append(anno_1[dimension][str(i+1)][model])
        oc[i]['human_anno'][model][0] = anno_1[dimension][str(i+1)][model]

with open(jsonpath, 'w') as f:
    json.dump(oc, f, indent=4)

In [6]:
# # 把GPT4o的结果写入excel文件 独立指标
# import json
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.styles import Border, Side


# models = ['cogvideox5b','gen3','kling','videocrafter2','pika','show1','lavie']
# jsonpath = '../Human_anno/{}.json'.format(dimension)
# with open(jsonpath, 'r') as f:
#     oc = json.load(f)

# ls = list(range(0, len(oc),3))


# df = pd.DataFrame(columns=['Prompt','Group','Model',dimension,'human1','human2','human3','lsy'])

# for index in ls:
#     prompt = oc[index]['prompt_en']
#     group = index%3
#     gptscore = []
#     lsy = []
#     h1 = []
#     h2 = []
#     h3  = []    
#     for model in models:
#         gptscore.append(oc[index]['gpt4o_eval'][model])
#         lsy.append(oc[index]['human_anno'][model][0])
#         h1.append(oc[index]['human_anno'][model][1])
#         h2.append(oc[index]['human_anno'][model][2])
#         h3.append(oc[index]['human_anno'][model][3])
#     new_row = pd.DataFrame({'Prompt': [prompt]*7, 'Group': [group]*7, 'Model': models, dimension: gptscore, 'human1': h1,'human2':h2,'human3':h3,'lsy': lsy})
#     df = pd.concat([df, new_row], ignore_index=True)



# output_path = '../Human_anno/humananno_res/xlsx/{}.xlsx'.format(dimension)
# df.to_excel(output_path,index=False)

# # 调整列宽
# wb = load_workbook(output_path)
# ws = wb.active


# for col in ws.columns:
#     sum = 0
#     column = col[0].column_letter  # 获取列字母
#     length = len(str(col[0].value))
#     for cell in col:
#         try:
#             sum += len(str(cell.value))
#         except:
#             pass
#     adjusted_width = max(sum/len(col) + 2,length+2)
#     ws.column_dimensions[column].width = adjusted_width

# # 添加分割线
# thin_border = Border(bottom=Side(style='thin'))
# for row in range(8, ws.max_row + 1, 7):  # 从第6行开始，每5行添加一次
#     for col in range(1, ws.max_column + 1):
#         cell = ws.cell(row=row, column=col)
#         cell.border = thin_border

# wb.save(output_path)


In [7]:
# #计算hum anno中完全相同的
# import json
# import pandas as pd
# import numpy as np

# models = ['cogvideox5b','gen3','kling','videocrafter2','pika','show1','lavie']
# jsonpath = '../Human_anno/color.json'
# with open(jsonpath, 'r') as f:
#     oc = json.load(f)

# samels = [0]*6
# for i in range(len(oc)):
#     human_anno = np.array(list(oc[i]['human_anno'].values()))
#     # baseline_rank = np.array(list(oc[i]['baseline_rank'].values())[3:])

#     lsy = human_anno[:,0]
#     anno1 = human_anno[:,1]
#     # anno2 = human_anno[:,1]
#     anno2 = human_anno[:,3]
#     if np.all(lsy == anno1):
#         samels[0] += 1
#     elif np.all(lsy == anno2):
#         samels[1] += 1
#     if np.all(anno1 == anno2):
#         samels[2] += 1
#     if np.sum(lsy == anno1) == 6:
#         samels[3] += 1
#     elif np.sum(lsy == anno2) == 6:
#         samels[4] += 1
#     elif np.sum(anno1 == anno2) == 6:
#         samels[5] += 1

# print(samels)


In [8]:
# # 把GPT4o的结果写入excel文件 独立指标
# import json
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.styles import Border, Side


# models = ['cogvideox5b','gen3','kling','videocrafter2','pika','show1','lavie']
# jsonpath = '../Human_anno/{}.json'.format(dimension)
# with open(jsonpath, 'r') as f:
#     oc = json.load(f)

# ls = list(range(0, len(oc),3))


# df = pd.DataFrame(columns=['Prompt','Group','Model',dimension,'human1','human2','human3','lsy'])

# for index in ls:
#     prompt = oc[index]['prompt_en']
#     group = index%3
#     gptscore = []
#     lsy = []
#     h1 = []
#     h2 = []
#     h3  = []    
#     for model in models:
#         gptscore.append(oc[index]['gpt4o_eval'][model])
#         lsy.append(oc[index]['human_anno'][model][0])
#         h1.append(oc[index]['human_anno'][model][1])
#         h2.append(oc[index]['human_anno'][model][2])
#         h3.append(oc[index]['human_anno'][model][3])
#     new_row = pd.DataFrame({'Prompt': [prompt]*7, 'Group': [group]*7, 'Model': models, dimension: gptscore, 'human1': h1,'human2':h2,'human3':h3,'lsy': lsy})
#     df = pd.concat([df, new_row], ignore_index=True)



# output_path = '../Human_anno/humananno_res/xlsx/{}.xlsx'.format(dimension)
# df.to_excel(output_path,index=False)

# # 调整列宽
# wb = load_workbook(output_path)
# ws = wb.active


# for col in ws.columns:
#     sum = 0
#     column = col[0].column_letter  # 获取列字母
#     length = len(str(col[0].value))
#     for cell in col:
#         try:
#             sum += len(str(cell.value))
#         except:
#             pass
#     adjusted_width = max(sum/len(col) + 2,length+2)
#     ws.column_dimensions[column].width = adjusted_width

# # 添加分割线
# thin_border = Border(bottom=Side(style='thin'))
# for row in range(8, ws.max_row + 1, 7):  # 从第6行开始，每5行添加一次
#     for col in range(1, ws.max_column + 1):
#         cell = ws.cell(row=row, column=col)
#         cell.border = thin_border

# wb.save(output_path)
